In [1]:
import clingo

In [2]:
def combine(encoding_path, benchmark_path, destination_path):
    
    combined_path = destination_path + "/combined.lp"
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    benchmark_file = open(benchmark_path, "r")
    benchmark = benchmark_file.read()
    benchmark_file.close()
    
    combined_file = open(combined_path, "w")
    combined = combined_file.write(benchmark + encoding)
    combined_file.close()

In [3]:
def solve(robot_id, destination_path, restrictions=[]):
    
    combined_path = destination_path + "/combined.lp"
    combined_file = open(combined_path, "r")
    combined = combined_file.read()
    combined_file.close()
    
    rid = "#const rid = {}.".format(robot_id)

    for i in range(1,100):
        solution = ""
        horizon = "#const horizon = {}.".format(i)
        asp = horizon + "\n" + rid + "\n" + combined
        if restrictions:
            for j in restrictions:
                if j:
                    asp = asp + "\n" + j
            
        ctl = clingo.Control()
        ctl.add("base", [], asp)
        ctl.ground([("base", [])])
        with ctl.solve(yield_=True) as handle:
            for m in handle: solution=("{} ".format(m))
            handle.get()
        solution = solution.replace(' ', '. ')
        
        if solution:
            print(i)
            return solution, i
            break

In [4]:
def collisions(robot_id, encoding_path, plan):
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    rid = "#const rid = {}.".format(robot_id)
    asp = rid + "\n" + plan + "\n" + encoding
    solution_ = ""
    
    ctl = clingo.Control()
    ctl.add("base", [], asp)
    ctl.ground([("base", [])])
    with ctl.solve(yield_=True) as handle:
        for m in handle: solution_=("{} ".format(m))
        handle.get()
    if solution_== " ":
        return
    else:
        solution_ = solution_.replace(' ', '. ')
        #print(solution)
        #print("\n")
        return solution_

In [13]:
def plan(robots, destination, collision, restrictions=[]):
    plans=[]
    plan_=""
    paths = []
    actions = []
    
    if not paths:
        for i in robots:
            paths.append([])

    for i in robots:
        temp = solve(i, destination, restrictions)
        paths[i-1].append(temp[0])
        actions.append(temp[1])
        print(actions)
        print("\n")
        if paths[i-1][0]:
            plan_ = plan_ + paths[i-1][0]
    
    act = sum(actions)
    new_act=act+1
    print(act)
    cur_rest = ["temp"]
    restrictions=[]
    
    while act < new_act and cur_rest[0]:
        act = new_act
        cur_rest = []
        cur_rest.append(collisions(1, collision, plan_))
        print(cur_rest)
        if cur_rest[0]:
            restrictions.append(cur_rest[-1])
        temp_ = replan(robots, destination, collision, restrictions)
        plan_ = temp[0]
        new_act = temp[1]
    return plan_

    """
    for i in robots:
        restrictions=[]
        cur_rest.append(collisions(i, collision, plan_))
        while cur_rest[0]:
            restrictions.append(cur_rest[-1])
            cur_rest = []
            plan_ = replan(robots, destination, collision, restrictions)
            cur_rest.append(collisions(i, collision, plan_))
        return plan_
        #plans.append(plan_)
    """
    #return plans

In [14]:
def replan(robots, destination, collision, restrictions=[]):
    plans=[]
    plan_=""
    paths = []
    actions = []
    
    if not paths:
        for i in robots:
            paths.append([])

    for i in robots:
        temp = solve(i, destination, restrictions)
        print(temp)
        print("\n")
        paths[i-1].append(temp[0])
        actions.append(temp[1])

        if paths[i-1][0]:
            plan_ = plan_ + paths[i-1][0]
    act = sum(actions)
    return plan_, act

In [15]:
def save_plans(plans, k=0):
    if type(plans)==list:
        for i in range(len(plans)):
            if type(plans[i])==str:
                with open("./python/solution_%d_%d.lp" % (k,i), 'w') as f:
                    f.write(plans[i])
            else:
                if type(plans[i])==list:
                    k+=1
                    save_plans(plans[i],k)
    else:
        if type(plans)==str:
                with open("./python/solution_%d.lp" % (k), 'w') as f:
                    f.write(plans)

In [16]:
enc = "./asprilo-encodings/shortest_path/pathfinder.lp"
ben = "./instances/example4/instance.lp"
dest = "./python"
col = "./asprilo-encodings/collision/collision.lp"
robots = []
for i in range(1,26):
    robots.append(i)

rest=[]    
plans =[]

In [17]:
combine(enc, ben, dest)

In [18]:
plans=plan(robots, dest, col, rest)

8
[8]


10
[8, 10]


5
[8, 10, 5]


14
[8, 10, 5, 14]


11
[8, 10, 5, 14, 11]


5
[8, 10, 5, 14, 11, 5]


9
[8, 10, 5, 14, 11, 5, 9]


17
[8, 10, 5, 14, 11, 5, 9, 17]


8
[8, 10, 5, 14, 11, 5, 9, 17, 8]


25
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25]


8
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8]


9
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9]


13
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9, 13]


10
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9, 13, 10]


12
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9, 13, 10, 12]


8
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9, 13, 10, 12, 8]


9
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9, 13, 10, 12, 8, 9]


7
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9, 13, 10, 12, 8, 9, 7]


5
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9, 13, 10, 12, 8, 9, 7, 5]


13
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9, 13, 10, 12, 8, 9, 7, 5, 13]


13
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9, 13, 10, 12, 8, 9, 7, 5, 13, 13]


23
[8, 10, 5, 14, 11, 5, 9, 17, 8, 25, 8, 9, 13, 10, 12, 8, 9, 7, 5, 13, 13, 23]


6

9
('position(12,(6,7),0). position(12,(7,7),1). position(12,(8,7),2). position(12,(9,7),3). position(12,(10,7),4). position(12,(11,7),5). position(12,(12,7),6). position(12,(13,7),7). position(12,(14,7),8). position(12,(14,8),9). occurs(object(robot,12),action(move,(1,0)),1). occurs(object(robot,12),action(move,(1,0)),2). occurs(object(robot,12),action(move,(1,0)),3). occurs(object(robot,12),action(move,(1,0)),4). occurs(object(robot,12),action(move,(1,0)),5). occurs(object(robot,12),action(move,(1,0)),6). occurs(object(robot,12),action(move,(1,0)),7). occurs(object(robot,12),action(move,(1,0)),8). occurs(object(robot,12),action(move,(0,1)),9). ', 9)


13
('position(13,(3,2),0). position(13,(3,3),1). position(13,(3,4),2). position(13,(3,5),3). position(13,(4,5),4). position(13,(4,6),5). position(13,(4,7),6). position(13,(4,8),7). position(13,(4,9),8). position(13,(4,10),9). position(13,(4,11),10). position(13,(4,12),11). position(13,(4,13),12). position(13,(4,14),13). occurs(object(rob

In [11]:
save_plans(plans)

In [12]:
print(plans)

position(25,(7,1),0). position(25,(7,2),1). position(25,(6,2),2). position(25,(6,3),3). position(25,(5,3),4). position(25,(4,3),5). position(25,(3,3),6). position(25,(3,4),7). position(25,(3,5),8). position(25,(3,6),9). position(25,(3,7),10). occurs(object(robot,25),action(move,(0,1)),1). occurs(object(robot,25),action(move,(-1,0)),2). occurs(object(robot,25),action(move,(0,1)),3). occurs(object(robot,25),action(move,(-1,0)),4). occurs(object(robot,25),action(move,(-1,0)),5). occurs(object(robot,25),action(move,(-1,0)),6). occurs(object(robot,25),action(move,(0,1)),7). occurs(object(robot,25),action(move,(0,1)),8). occurs(object(robot,25),action(move,(0,1)),9). occurs(object(robot,25),action(move,(0,1)),10). 


In [ ]:
to do:
    solve
    collisions
    plan
    replan

In [ ]:
plan -> solve for every robot -> collisions (get restrictions) -> replan (solve with restrictions)